<a href="https://colab.research.google.com/github/regenbuild/BIM2BEM/blob/main/TopologicWallTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install topologicpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.0/292.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 51.4 MB/s eta 0:00:00


In [3]:
class Wall:
    """
    The wall class represents topology of a single wall which can be vertices, faces, or cells
    """
    def __init__(self, vertices):
        self.all_vertices = vertices
        self.filtered_vertices = vertices
        self.sorted_vertices = []
        self.after_sort_vertices = []
        self.faces = []
        self.cell = None

    def __str__(self):
        data = [[each.X(), each.Y(), each.Z()] for each in self.sorted_vertices]
        res = f"   X      Y      Z\n"
        for i in range(len(data)):
            res += f"{i}) {data[i][0]}, {data[i][1]}, {data[i][2]}\n"
        return res

    def keep_8_coords(self):
        """
        Method that addreses a case when wall has openings like wall or window
        Method reduces amount of vertices to 8 which represent boundings
        """
        if len(self.all_vertices) > 8:
            centroid = Vertex.Centroid(self.all_vertices)
            self.filtered_vertices = sorted(self.all_vertices, key=lambda x: Vertex.Distance(centroid, x), reverse=True)[0:8]
        else:
            self.filtered_vertices = self.all_vertices

    def sort_vertices(self):
        """
        Method that addresses the problem of disorder of vertices
        It sorted all of the wall vertices clockwise
        """
        def find_min_max(list_of_coords):
            z_list = [i.Z() for i in list_of_coords]
            return  [min(z_list), max(z_list)]

        only_min = [i for i in self.filtered_vertices if i.Z() == find_min_max(self.filtered_vertices)[0]]
        only_max = [i for i in self.filtered_vertices if i.Z() == find_min_max(self.filtered_vertices)[1]]
        clockwise_min =  Vertex.Clockwise2D(only_min)
        clockwise_max =  Vertex.Clockwise2D(only_max)

        res = []
        for each in list(zip(clockwise_min, clockwise_max)):
            res.extend(each)

        self.sorted_vertices = res

    def after_sort(self):
        """
        Method that prepares the points for faces by making groups of 4 vertices
        """
        coords_for_faces = []
        coords_for_faces.append([self.sorted_vertices[0], self.sorted_vertices[1], self.sorted_vertices[3], self.sorted_vertices[2]])
        coords_for_faces.append([self.sorted_vertices[2], self.sorted_vertices[3], self.sorted_vertices[5], self.sorted_vertices[4]])
        coords_for_faces.append([self.sorted_vertices[4], self.sorted_vertices[5], self.sorted_vertices[7], self.sorted_vertices[6]])
        coords_for_faces.append([self.sorted_vertices[6], self.sorted_vertices[7], self.sorted_vertices[1], self.sorted_vertices[0]])
        coords_for_faces.append([self.sorted_vertices[0], self.sorted_vertices[2], self.sorted_vertices[4], self.sorted_vertices[6]])
        coords_for_faces.append([self.sorted_vertices[1], self.sorted_vertices[3], self.sorted_vertices[5], self.sorted_vertices[7]])

        self.after_sort_coordinates = coords_for_faces

    def make_faces(self):
        faces_list = []
        for each in self.after_sort_coordinates:
            face = Face.ByVertices(each)
            faces_list.append(face)

        self.faces = faces_list

    def make_cell(self):
        self.cell = Cell.ByFaces(self.faces)

    def show_cell_topology(self):
        Topology.Show(self.cell, renderer='browser')

In [4]:
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Topology import Topology
from topologicpy.Graph import Graph
from topologicpy.Dictionary import Dictionary
from topologicpy.Plotly import Plotly
from topologicpy.Vertex import Vertex
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell


## WALL1
vertex_1 = Vertex.ByCoordinates(0, 0, 0)
vertex_2 = Vertex.ByCoordinates(0, 10, 0)
vertex_3 = Vertex.ByCoordinates(2, 10, 0)
vertex_4 = Vertex.ByCoordinates(2, 0, 0)
vertex_5 = Vertex.ByCoordinates(0, 0, 5)
vertex_6 = Vertex.ByCoordinates(0, 10, 5)
vertex_7 = Vertex.ByCoordinates(2, 10, 5)
vertex_8 = Vertex.ByCoordinates(2, 0, 5)

wall_1_vertices_list = [vertex_1, vertex_2, vertex_3, vertex_4, vertex_5, vertex_6, vertex_7, vertex_8]
wall_1 = Wall(wall_1_vertices_list)
wall_1.sort_vertices()
wall_1.after_sort()
wall_1.make_faces()

## WALL2
w_2_vertex_1 = Vertex.ByCoordinates(8, 0, 0)
w_2_vertex_2 = Vertex.ByCoordinates(8, 10, 0)
w_2_vertex_3 = Vertex.ByCoordinates(10, 10, 0)
w_2_vertex_4 = Vertex.ByCoordinates(10, 0, 0)
w_2_vertex_5 = Vertex.ByCoordinates(8, 0, 5)
w_2_vertex_6 = Vertex.ByCoordinates(8, 10, 5)
w_2_vertex_7 = Vertex.ByCoordinates(10, 10, 5)
w_2_vertex_8 = Vertex.ByCoordinates(10, 0, 5)

wall_2_vertices_list = [w_2_vertex_1, w_2_vertex_2, w_2_vertex_3, w_2_vertex_4, w_2_vertex_5, w_2_vertex_6, w_2_vertex_7, w_2_vertex_8]
wall_2 = Wall(wall_2_vertices_list)
wall_2.sort_vertices()
wall_2.after_sort()
wall_2.make_faces()

## WALL3
w_3_vertex_1 = Vertex.ByCoordinates(0, 0, 0)
w_3_vertex_2 = Vertex.ByCoordinates(0, 2, 0)
w_3_vertex_3 = Vertex.ByCoordinates(10, 2, 0)
w_3_vertex_4 = Vertex.ByCoordinates(10, 0, 0)
w_3_vertex_5 = Vertex.ByCoordinates(0, 0, 5)
w_3_vertex_6 = Vertex.ByCoordinates(0, 2, 5)
w_3_vertex_7 = Vertex.ByCoordinates(10, 2, 5)
w_3_vertex_8 = Vertex.ByCoordinates(10, 0, 5)

wall_3_vertices_list = [w_3_vertex_1, w_3_vertex_2, w_3_vertex_3, w_3_vertex_4, w_3_vertex_5, w_3_vertex_6, w_3_vertex_7, w_3_vertex_8]
wall_3 = Wall(wall_3_vertices_list)
wall_3.sort_vertices()
wall_3.after_sort()
wall_3.make_faces()

## WALL3
w_4_vertex_1 = Vertex.ByCoordinates(0, 8, 0)
w_4_vertex_2 = Vertex.ByCoordinates(0, 10, 0)
w_4_vertex_3 = Vertex.ByCoordinates(10, 10, 0)
w_4_vertex_4 = Vertex.ByCoordinates(10, 8, 0)
w_4_vertex_5 = Vertex.ByCoordinates(0, 8, 5)
w_4_vertex_6 = Vertex.ByCoordinates(0, 10, 5)
w_4_vertex_7 = Vertex.ByCoordinates(10, 10, 5)
w_4_vertex_8 = Vertex.ByCoordinates(10, 8, 5)

wall_4_vertices_list = [w_4_vertex_1, w_4_vertex_2, w_4_vertex_3, w_4_vertex_4, w_4_vertex_5, w_4_vertex_6, w_4_vertex_7, w_4_vertex_8]
wall_4 = Wall(wall_4_vertices_list)
wall_4.sort_vertices()
wall_4.after_sort()
wall_4.make_faces()



Topology.Show(wall_1.faces, wall_2.faces, wall_3.faces, wall_4.faces, renderer="colab")